<a href="https://colab.research.google.com/github/lamaT5/T5./blob/main/Week5_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df=pd.read_csv('/content/Traffic Data.csv' , parse_dates=['date_time'] , index_col=['date_time'])

df.sort_index(inplace=True)

In [ ]:
df.info()

In [ ]:
df.head()

### **EDA**

In [ ]:
df.plot()

In [ ]:
#outlire (boxplot)

import seaborn as sns
sns.boxplot(df)

In [ ]:
import matplotlib.pyplot as plt

df['temp'].plot()
plt.title('temperature Average')
plt.xlabel('date time')
plt.ylabel('num')

In [ ]:
import matplotlib.pyplot as plt

df['rain_1h'].plot()

In [ ]:
sns.countplot(df , x = 'weather_main')

In [ ]:
df['traffic_volume'].plot()
plt.title('Traffic_Volume')
plt.xlabel('data time')

In [ ]:
df.isnull().sum()

### **Data Preprocessing**

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df.isnull().sum()

# **Feature Engineering**

In [ ]:
#shift (traffic volume)
df['prev_traffic_volume'] = df['traffic_volume'].shift(1)
df[['traffic_volume' , 'prev_traffic_volume']]

In [ ]:
# diff
df['diff_traffic_volume'] = df['traffic_volume'].diff()
df[['traffic_volume' , 'diff_traffic_volume']]

**Date Features**: Extract meaningful time-based features from the date_time column to enhance the model's ability to detect temporal patterns. These features include:

**Day of the week:** To capture weekly seasonality.
**Month:** To capture monthly trends.
**Day of the year**: Useful for modeling yearly seasonality

In [ ]:
df['Day_of_the_week'] = df.index.dayofweek

In [ ]:
df['Month'] = df.index.month

In [ ]:
df['Day_of_the_year'] = df.index.dayofyear

In [ ]:
df[['Day_of_the_week' , 'Month' , 'Day_of_the_year']]

**Feature Scaling**
Apply feature scaling to the data if needed

**Instructions:**

Use a scaling method such as MinMaxScaler or StandardScaler.
Ensure scaling is applied correctly to both training and testing data

In [ ]:
target = df['traffic_volume'].values.reshape(-1,1)

In [ ]:
df['traffic_volume'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
# calclates mean
scaler = StandardScaler()
target_pre = scaler.fit_transform(target)

**Train-Test Split**

In [ ]:
train_size = int(len(target_pre)*0.8)
train , test = target_pre[:train_size] , target_pre[train_size:]

In [ ]:
x_train , y_train = train[:-1] , train[1:]
x_test , y_test = test[:-1] , test[1:]

In [ ]:
x_train = x_train.reshape(-1,1,1)
x_test = x_test.reshape(-1,1,1)

In [ ]:
x_train.shape , x_test.shape

## **Models Selection**

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,LSTM,SimpleRNN,Dropout

In [ ]:

model = Sequential([

    SimpleRNN(16 , activation='tanh' , return_sequences=True),
    Dropout(0.2),

    SimpleRNN(32 , activation='tanh' , return_sequences=True),
    Dropout(0.2),

    Dense(1)

                    ])

In [ ]:
model.compile(optimizer='adam', loss='mae', metrics=['mae'])


In [ ]:
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=1024)


In [ ]:
y_pre = model.predict(x_test)

y_pre = y_pre.reshape(y_pre.shape[0], -1)


In [ ]:
y_test = scaler.inverse_transform(y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


mae = mean_absolute_error(y_test, y_pre)
mse = mean_squared_error(y_test, y_pre)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pre)

print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)
print('MAPE:', mape)



In [ ]:
plt.title('Actual vs predict')
plt.plot(y_test)
plt.plot(train_pred)
plt.show()